<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/semaforos_do_recife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='40%' aling='center' src="https://github.com/leonarddocchagas/data_science/blob/master/projetos/semaforos_do_recife/logo_recife.jpg?raw=true" />

A **Autarquia de Trânsito e Transporte Urbano do Recife (CTTU)** é uma sociedade de economia mista integrante da Administração Indireta Municipal e é responsável pelo monitoramento, controle e fiscalização do trânsito e do transporte na capital. Compete à CTTU também a regulamentação do Sistema Municipal de Transporte Público de Passageiros, que é explorado sob a forma de permissão ou autorização nos modais de táxi, transporte complementar (linhas interbairros e alimentadoras) e transporte escolar, ações de estudos e projetos de melhoria da mobilidade urbana e sua efetiva implantação e de educação para o trânsito.

*Fonte:* https://bit.ly/3svji3h

##Sobre os dados

Relação de todos os semáforos georreferenciados, contendo informações referentes ao modo de funcionamento, e tipo de semáforo.
  * Dados coletados do http://dados.recife.pe.gov.br/
    * Dataset: https://bit.ly/2XGmA8u
    * Dicionário: https://bit.ly/3kcfBLQ
    * Mais: https://bit.ly/3y20D02
  
  * Lista das variáveis:
    * _id
    * semaforo
    * localizacao1
    * localizacao2
    * bairro
    * latitude
    * longitude
    * tipo

##Obtenção e transformação dos dados

**Carrega e verifica dataset**

In [33]:
# bibliotecas
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

# criar mapa
#!pip install folium
import folium
from folium import plugins

In [30]:
PATH_DATA = 'http://dados.recife.pe.gov.br/dataset/ab87d3e2-ea78-4f27-95fd-d7ef6f3b9ee5/resource/ab6343e9-c3f2-4d62-9554-5778f9f33738/download/semaforos_bairro.csv'
df = pd.read_csv(PATH_DATA, error_bad_lines=False, sep=';')
df.head()

,semafororo,localizacao1,localizacao2,bairro,latitude,longitude,tipo,funcionamento
0,1,Av. Boa Viagem,Em frente ao parque Dona Lindú,Boa Viagem,-8.142323,-34.903798,E/Git,Ocasional/Ped.
1,2,Av. Conselheiro Aguiar,Rua Thomé Gibson,Pina,-8.099068,-34.885883,E/Git,Veicular
2,3,Av. Boa Viagem,Rua França Pereira,Boa Viagem,-8.106806,-34.887726,E/Git,Ocasional/Ped.
3,4,Av. Boa Viagem,Rua Barão de Souza Leão,Boa Viagem,-8.132230,-34.899990,E/Git,Ocasional/Ped.
4,5,Av. Conselheiro Aguiar,Rua Artur Muniz,Boa Viagem,-8.104310,-34.888228,E/Git,Veicular


In [31]:
df['bairro'].value_counts()

Boa Viagem            97
Imbiribeira           43
Santo Amaro           40
Santo Antônio         27
São José              26
                      ..
Ilha Joana Bezerra     1
Coelhos                1
Alto José do Pinho     1
Curado                 1
Linha do Tiro          1
Name: bairro, Length: 75, dtype: int64

In [51]:
coordenadas=[]
for lat, lng in zip(df.latitude, df.longitude):
  coordenadas.append([lat, lng])

In [52]:
mapa = folium.Map(
    location=[-8.0704276,-34.894891], 
    zoom_start=7, 
    tiles='OpenStreetMap' # tipo do mapa: Stamen Toner
    )

# exibi mapa
mapa.add_child(plugins.HeatMap(coordenadas))